# Simulation template

In this notebook we run the machine learning analysis of topological phase transitions occurring  in both nearest-neighbours SSH models (ssh1) and second neighbours models (ssh2) as decribed in the paper [Machine learning topological phases in real space](https://arxiv.org/abs/1901.01963).

## Defining parameters

In [1]:
%%time
%load_ext autoreload
%autoreload 2
from simulation import *

CPU times: user 742 ms, sys: 276 ms, total: 1.02 s
Wall time: 572 ms


In [2]:
%%time
### Dataset and simulation parameters
csv_path = SSH1_PERIODIC_100_6561_CSV 
model_name = "DecisionTreeClassifier"
model_kw = {"criterion":"entropy"}
allowed_windings = [0,1]
simulation_dir = SSH1_PERIODIC_100_6561_SIMULATION_DIR
val_split = 0.9  
features_to_use = None 
shuffle_features = False
random_state = 137                    

### Running a simulation
n_experiments = 5
start_n = 0
fit_params = None
shuffle_rows = True
pred_params = None
random_features = False
######### DON'T SET THIS TO TRUE UNLESS YOUR DATASET IS SMALL!! WILL FLOOD YOUR MEMORY!!!
store_in_lists = False   
########## BELOW ARE THE PARAMETERS THAT CONTROL WHAT WILL BE SAVED
save_eigenvector=True
save_hamiltonian=True 
save_accuracy=True 
save_models=True

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 5.48 µs


In [3]:
# Parameters
model_kw = {"criterion": "entropy", "n_estimators": 25, "n_jobs": -1}
allowed_windings = [-1, 0, 1, 2]
val_split = 0.5
features_to_use = None
shuffle_features = False
n_experiments = 100
start_n = 0
fit_params = None
shuffle_rows = True
pred_params = None
random_features = False
store_in_lists = False
save_eigenvector = True
save_hamiltonian = True
save_accuracy = True
save_models = True
csv_path = "/home/rio/ssh_csvs/ssh2/periodic_180_6561.csv"
model_name = "RandomForestClassifier"
simulation_dir = "/home/rio/ssh_simulations/ssh2/periodic_180_6561"
random_state = 11


In [4]:
%%time
#Starting an instance of Simulation with a decision tree
model = DecisionTreeClassifier(criterion="entropy")
#dict_args = {"csv_path":csv_path, "model":model, "allowed_windings":allowed_windings,\
#             "simulation_dir":simulation_dir, "val_split":val_split, "features_to_use":features_to_use,\
#            "shuffle_features":shuffle_features, "random_state":random_state}
simulation = Simulation(csv_path,model_name,model_kw,allowed_windings,simulation_dir,val_split,features_to_use,\
                       shuffle_features,random_state)
#simulation = Simulation(**dict_args)

print("Info on all data: \n")
simulation.dataframe.info()
simulation.dataframe.head()

/home/rio/ml_top_phases/lib/python3.6/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


Info on all data: 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1180980 entries, 0 to 1180979
Columns: 188 entries, id to feat179
dtypes: float64(183), int32(3), object(2)
memory usage: 1.6+ GB
CPU times: user 16.9 s, sys: 787 ms, total: 17.7 s
Wall time: 17.7 s


,id,path,t1,t2,winding,phase,pred_phase,type_of,feat0,feat1,...,feat170,feat171,feat172,feat173,feat174,feat175,feat176,feat177,feat178,feat179
0,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,666,test,-0.130846,-0.300844,...,-0.080274,0.115552,-0.174807,-0.244872,0.319478,0.059064,-0.000568,0.244892,-0.300844,-0.130846
1,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,666,test,-0.130846,0.300844,...,-0.080274,-0.115552,-0.174807,0.244872,0.319478,-0.059064,-0.000568,-0.244892,-0.300844,0.130846
2,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,666,test,-0.146579,0.043053,...,0.187020,0.091249,-0.074116,0.114272,-0.147971,-0.194939,0.194123,0.055547,-0.043053,0.146579
3,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,666,test,0.013027,-0.001277,...,-0.007714,0.018139,-0.013208,-0.022654,0.017319,0.006334,-0.006175,0.006953,0.001277,-0.013027
4,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,666,test,0.003440,0.011453,...,-0.015015,-0.019046,0.015313,0.003978,-0.008323,0.008902,-0.003899,-0.013061,0.011453,0.003440


#### Checking initialization

In [5]:
%%time
n_features = simulation.n_features
n_hamiltonians = simulation.n_hamiltonians
n_ts = simulation.n_ts

print("n_features: ", n_features)
print("n_hamiltonians: ", n_hamiltonians)
print("n_ts: ", n_ts)

n_features:  180
n_hamiltonians:  6561
n_ts:  2
CPU times: user 193 µs, sys: 12 µs, total: 205 µs
Wall time: 153 µs


In [6]:
%%time
n_total = len(simulation.dataframe)
n_train = len(simulation.dataframe[simulation.dataframe.type_of == "train"])
n_val = len(simulation.dataframe[simulation.dataframe.type_of == "val"])
n_test = len(simulation.dataframe[simulation.dataframe.type_of == "test"])
n_train_hamiltonians = len(simulation.train_ids)
n_val_hamiltonians = len(simulation.val_ids)
n_test_hamiltonians = len(simulation.test_ids)
n_total_hamiltonians = n_train_hamiltonians + n_val_hamiltonians + n_test_hamiltonians

print("% train: ", n_train/n_total)
print("% val: ",  n_val/n_total)
print("% test: ", n_test/n_total)
print("% train + val + test: ", (n_train+n_val+n_test)/n_total)
print("\n")
print("number of train hamiltonians: ", n_train_hamiltonians)
print("number of val hamiltonians: ", n_val_hamiltonians)
print("number of test hamiltonians: ", n_test_hamiltonians)
print("total number of hamiltonians: ", n_total_hamiltonians)
print("\n")
print("train ids: ", simulation.train_ids)
print("val ids: ", simulation.val_ids)
print("test ids: ", simulation.test_ids)

% train:  0.8414875781130925
% val:  0.0
% test:  0.15851242188690748
% train + val + test:  1.0


number of train hamiltonians:  5521
number of val hamiltonians:  0
number of test hamiltonians:  1040
total number of hamiltonians:  6561


train ids:  [13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 201, 202, 203, 204, 205, 206, 207, 

## Running a simulation

In [7]:
%%time
simulation.run_simulation(n_experiments, start_n, fit_params, shuffle_rows, pred_params, random_features, \
                          store_in_lists, save_eigenvector, save_hamiltonian, save_accuracy,\
                          save_models)

simulation.dataframe.head(10)

running experiments:   0%|          | 0/100 [00:00<?, ?it/s]

THIS IS  predict_params:  None
THIS IS  the type of predict_params:  <class 'NoneType'>


running experiments:   1%|          | 1/100 [01:50<3:02:29, 110.60s/it]

THIS IS  predict_params:  None
THIS IS  the type of predict_params:  <class 'NoneType'>


running experiments:   2%|▏         | 2/100 [03:43<3:01:45, 111.28s/it]

...

THIS IS  predict_params:  None
THIS IS  the type of predict_params:  <class 'NoneType'>


running experiments: 100%|██████████| 100/100 [2:59:03<00:00, 107.15s/it]

running experiments: 100%|██████████| 100/100 [2:59:03<00:00, 107.43s/it]

CPU times: user 20h 8min 59s, sys: 47 s, total: 20h 9min 46s
Wall time: 2h 59min 3s


,id,path,t1,t2,winding,phase,pred_phase,type_of,feat0,feat1,...,feat170,feat171,feat172,feat173,feat174,feat175,feat176,feat177,feat178,feat179
0,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,-1,test,-0.130846,-0.300844,...,-0.080274,0.115552,-0.174807,-0.244872,0.319478,0.059064,-0.000568,0.244892,-0.300844,-0.130846
1,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,-1,test,-0.130846,0.300844,...,-0.080274,-0.115552,-0.174807,0.244872,0.319478,-0.059064,-0.000568,-0.244892,-0.300844,0.130846
2,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,2,test,-0.146579,0.043053,...,0.187020,0.091249,-0.074116,0.114272,-0.147971,-0.194939,0.194123,0.055547,-0.043053,0.146579
3,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,2,test,0.013027,-0.001277,...,-0.007714,0.018139,-0.013208,-0.022654,0.017319,0.006334,-0.006175,0.006953,0.001277,-0.013027
4,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,-1,test,0.003440,0.011453,...,-0.015015,-0.019046,0.015313,0.003978,-0.008323,0.008902,-0.003899,-0.013061,0.011453,0.003440
5,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,1,test,0.018047,0.037394,...,0.029254,0.015857,0.004441,0.027159,-0.041982,-0.021236,0.005203,-0.024027,0.037394,0.018047
6,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,1,test,0.084825,-0.026222,...,-0.104377,-0.066035,0.055706,-0.051816,0.080410,0.116916,-0.119805,-0.040657,0.026222,-0.084825
7,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,2,test,-0.043223,0.005021,...,0.026354,-0.048356,0.035233,0.067210,-0.054791,-0.024377,0.024692,-0.018139,-0.005021,0.043223
8,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,0,test,-0.009349,-0.031973,...,0.043557,0.051909,-0.042604,-0.009129,0.022361,-0.026100,0.011385,0.036684,-0.031973,-0.009349
9,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,0,test,0.028827,0.057672,...,0.053523,0.039631,-0.002767,0.038861,-0.065675,-0.039973,0.010875,-0.033361,0.057672,0.028827


#### Viewing a random sample

In [8]:
%%time
simulation.dataframe.sample(frac=0.1, replace=False)

CPU times: user 168 ms, sys: 0 ns, total: 168 ms
Wall time: 168 ms


,id,path,t1,t2,winding,phase,pred_phase,type_of,feat0,feat1,...,feat170,feat171,feat172,feat173,feat174,feat175,feat176,feat177,feat178,feat179
465980,2588,H_-0.45_1.85_2.00106.dat,-0.45,1.85,2.001061,2,2,val,0.061165,-0.001752,...,-0.111049,0.096261,0.129452,-0.066034,-0.057607,0.078318,0.023319,-0.111963,-0.001752,0.061165
512992,2849,H_-0.25_-1.3_1.00398.dat,-0.25,-1.30,1.003977,1,1,val,0.001247,0.017194,...,0.019981,0.057884,-0.008891,-0.047305,-0.002760,0.037835,0.015285,-0.025908,-0.017194,-0.001247
974149,5411,H_1.3_1.25_-1.06092.dat,1.30,1.25,-1.060924,-1,-1,test,-0.105403,0.028964,...,-0.075635,0.142349,-0.023921,-0.098034,0.034255,-0.001151,0.065263,-0.058182,-0.028964,0.105403
529876,2943,H_-0.2_-0.65_1.00603.dat,-0.20,-0.65,1.006026,1,1,val,-0.010562,0.061565,...,0.107220,-0.012877,-0.083594,-0.051012,-0.001315,0.115923,0.043715,-0.077390,-0.061565,0.010562
1124386,6246,H_1.85_-1.55_1.00072.dat,1.85,-1.55,1.000718,1,1,train,-0.031869,-0.038062,...,-0.003029,-0.048306,-0.096820,-0.100935,-0.022646,0.015268,0.088684,0.088747,0.038062,0.031869
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
802728,4459,H_0.75_-1.8_1.00164.dat,0.75,-1.80,1.001638,1,1,val,0.048231,0.017515,...,0.067427,0.126849,-0.131988,-0.021757,0.031712,0.125777,-0.114544,-0.068658,0.017515,0.048231
335494,1863,H_-0.85_-2_2.00552.dat,-0.85,-2.00,2.005517,2,2,train,0.004323,-0.000935,...,-0.016309,-0.048864,-0.008676,-0.037189,-0.003712,-0.025002,-0.001033,-0.013991,0.000935,-0.004323
407296,2262,H_-0.65_1.75_2.00084.dat,-0.65,1.75,2.000836,2,2,val,0.029485,0.018131,...,0.001862,-0.023607,-0.025095,-0.017183,-0.046971,0.019444,-0.010629,0.035591,0.018131,0.029485
766737,4259,H_0.6_0.35_0.998698.dat,0.60,0.35,0.998698,1,1,train,-0.024686,0.047075,...,-0.113562,0.120074,0.103058,-0.104027,-0.043446,0.047800,-0.020854,0.000097,0.047075,-0.024686


#### Checking train/val/test splits again

In [9]:
%%time
n_total = len(simulation.dataframe)
n_train = len(simulation.dataframe[simulation.dataframe.type_of == "train"])
n_val = len(simulation.dataframe[simulation.dataframe.type_of == "val"])
n_test = len(simulation.dataframe[simulation.dataframe.type_of == "test"])
n_train_hamiltonians = len(simulation.train_ids)
n_val_hamiltonians = len(simulation.val_ids)
n_test_hamiltonians = len(simulation.test_ids)
n_total_hamiltonians = n_train_hamiltonians + n_val_hamiltonians + n_test_hamiltonians

print("% train: ", n_train/n_total)
print("% val: ",  n_val/n_total)
print("% test: ", n_test/n_total)
print("% train + val + test: ", (n_train+n_val+n_test)/n_total)
print("\n")
print("number of train hamiltonians: ", n_train_hamiltonians)
print("number of val hamiltonians: ", n_val_hamiltonians)
print("number of test hamiltonians: ", n_test_hamiltonians)
print("total number of hamiltonians: ", n_total_hamiltonians)
print("\n")
print("train ids: ", simulation.train_ids)
print("val ids: ", simulation.val_ids)
print("test ids: ", simulation.test_ids)

% train:  0.4208199969516842
% val:  0.42066758116140834
% test:  0.15851242188690748
% train + val + test:  1.0


number of train hamiltonians:  2761
number of val hamiltonians:  2760
number of test hamiltonians:  1040
total number of hamiltonians:  6561


train ids:  [1876, 4475, 6373, 4905, 2095, 2644, 3840, 5219, 145, 4502, 6186, 4821, 2287, 273, 1157, 6161, 27, 2834, 1913, 3242, 4791, 4270, 139, 3903, 3231, 4767, 2809, 2205, 2968, 2614, 3813, 3465, 5594, 5272, 5948, 77, 4976, 3144, 3842, 6122, 4762, 449, 4042, 4311, 6539, 6523, 2285, 436, 598, 4761, 5850, 869, 5462, 6372, 3003, 3369, 866, 4618, 4804, 2843, 1100, 789, 2732, 689, 71, 4412, 6465, 47, 1942, 4460, 5949, 2781, 5488, 2526, 302, 5897, 5100, 4396, 3836, 296, 3001, 1367, 6107, 5881, 3424, 4612, 5433, 606, 3785, 6139, 379, 2899, 4286, 3092, 3791, 2265, 6412, 4833, 6136, 6239, 6017, 1591, 3431, 4786, 5155, 2586, 2484, 1515, 796, 597, 1924, 2152, 4024, 3384, 607, 1514, 1419, 6494, 3337, 3051, 4937, 3649, 208, 5676, 2493, 3226,

#### Checking summaries

In [10]:
%%time
ham_summary = simulation.hamiltonian_summary
print("length of ham_summary: ", len(ham_summary))
ham_summary

length of ham_summary:  6561
CPU times: user 106 µs, sys: 0 ns, total: 106 µs
Wall time: 84.9 µs


,t1,t2,type_of,-1,0,1,2,phase,pred_phase
id,,,,,,,,,
0,-2.0,-2.00,test,0.205556,0.383333,0.244444,0.166667,999,0
1,-2.0,-1.95,test,0.238889,0.394444,0.222222,0.144444,-1,0
2,-2.0,-1.90,test,0.255556,0.338889,0.255556,0.150000,-1,0
3,-2.0,-1.85,test,0.233333,0.361111,0.266667,0.138889,-1,0
4,-2.0,-1.80,test,0.294444,0.377778,0.233333,0.094444,-1,0
...,...,...,...,...,...,...,...,...,...
6556,2.0,1.80,test,0.422222,0.222222,0.116667,0.238889,-1,-1
6557,2.0,1.85,test,0.405556,0.216667,0.083333,0.294444,-1,-1
6558,2.0,1.90,test,0.366667,0.200000,0.116667,0.316667,-1,-1


In [11]:
eigen_summary = simulation.eigenvector_summary
print("length of ham_summary: ", len(eigen_summary))
eigen_summary

length of ham_summary:  1180980


,id,phase,pred_phase,type_of
0,0,999,-1,test
1,0,999,-1,test
2,0,999,2,test
3,0,999,2,test
4,0,999,-1,test
...,...,...,...,...
1180975,6560,999,0,test
1180976,6560,999,0,test
1180977,6560,999,0,test
1180978,6560,999,0,test


#### Checking accuracies

In [12]:
simulation.accuracy

{'eigenvector_train': 0.999786711738903,
 'eigenvector_val': 0.9576871980676328,
 'eigenvector_test': 0.6415470184969603,
 'hamiltonian_train': 1.0,
 'hamiltonian_val': 0.9945652173913043,
 'hamiltonian_test': 0.8952270081490105}

#### Checking data stored in  memory

In [13]:
ham_summary_list = simulation.hamiltonian_summary_list
ham_summary_list

[]

In [14]:
eigen_summary_list = simulation.eigenvector_summary_list
eigen_summary_list

[]

In [15]:
accuracy_list = simulation.accuracy_list
accuracy_list

{'eigenvector_train': [],
 'eigenvector_val': [],
 'eigenvector_test': [],
 'hamiltonian_train': [],
 'hamiltonian_val': [],
 'hamiltonian_test': []}